In [41]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import joblib
import numpy as np

# ==============================================================================
# BAGIAN I: PERSIAPAN DATA, PELATIHAN, DAN PENYIMPANAN MODEL SVR
# ==============================================================================
print("--- MEMULAI BAGIAN I: PELATIHAN DAN PENYIMPANAN MODEL SVR ---")

# --- 1. Muat Dataset ---
try:
    df = pd.read_csv('food_data_final_shuffled.csv')
    print("Dataset 'food_data_final_shuffled.csv' berhasil dimuat.")
except FileNotFoundError:
    print("Error: File 'food_data_final_shuffled.csv' tidak ditemukan.")
    exit()

# --- 2. Definisikan Fitur Input (X) dan Target Output (y) ---
features = ['food_type', 'portion_size']
X = df[features]
targets = ['carbohydrates', 'protein', 'fat', 'calories']
y = df[targets]
print(f"\nFitur Input (X): {features}")
print(f"Target Output (y): {targets}")

# --- 3. Pra-pemrosesan Fitur Input (X) ---
X_encoded = pd.get_dummies(X, columns=['food_type'])
feature_names = X_encoded.columns.tolist()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# --- 4. Split Data ---
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

# --- 5. Latih Model Multi-Output SVR ---
print("\n--- Training Multi-Output SVR ---")
base_svr = SVR(kernel='rbf', C=400, gamma='scale', epsilon=0.05)
svr_model = MultiOutputRegressor(base_svr, n_jobs=-1) 
svr_model.fit(X_train_scaled, y_train)
print("SVR training complete.")

# --- 6. Evaluasi Model SVR ---
print("\n--- Mengevaluasi Model SVR ---")
y_pred_svr = svr_model.predict(X_test_scaled)
results = {}
for i, target_name in enumerate(targets):
    y_test_col = y_test.iloc[:, i]
    y_pred_col = y_pred_svr[:, i]
    mape = np.nan
    if not np.any(y_test_col == 0):
        mape = mean_absolute_percentage_error(y_test_col, y_pred_col)
    results[target_name] = {
        'MAE': mean_absolute_error(y_test_col, y_pred_col),
        'RMSE': np.sqrt(mean_squared_error(y_test_col, y_pred_col)),
        'R2 Score': r2_score(y_test_col, y_pred_col),
        'MAPE': mape
    }
results_df = pd.DataFrame(results).T
results_df['MAPE'] = results_df['MAPE'].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else "N/A")
print("\nHasil Evaluasi SVR per Target Nutrisi:")
print(results_df)

# --- 7. Simpan Model dan Komponennya ---
print("\n--- Menyimpan model SVR, scaler, dan nama fitur ---")
joblib.dump(svr_model, 'svr_multi_output_model.pkl')
joblib.dump(scaler, 'svr_scaler.pkl')
joblib.dump(feature_names, 'svr_feature_names.pkl')
print("Model dan komponen berhasil disimpan ke dalam file .pkl")


# ==============================================================================
# BAGIAN II: SIMULASI PENGGUNAAN MODEL YANG DISIMPAN
# ==============================================================================
print("\n\n--- MEMULAI BAGIAN II: PREDIKSI PADA DATA BARU ---")

def predict_nutrition_from_portion(food_type_str: str, portion_size_int: int):
    """
    Fungsi untuk memuat model SVR yang tersimpan dan melakukan prediksi nutrisi lengkap
    berdasarkan jenis makanan dan ukuran porsi.
    """
    try:
        model = joblib.load('svr_multi_output_model.pkl')
        scaler = joblib.load('svr_scaler.pkl')
        feature_names_list = joblib.load('svr_feature_names.pkl')
    except FileNotFoundError:
        print("Error: Salah satu file model (.pkl) tidak ditemukan. Jalankan bagian I terlebih dahulu.")
        return None

    new_data = pd.DataFrame([{'food_type': food_type_str, 'portion_size': portion_size_int}])
    
    new_data_encoded = pd.get_dummies(new_data)
    new_data_reindexed = new_data_encoded.reindex(columns=feature_names_list, fill_value=0)
    new_data_scaled = scaler.transform(new_data_reindexed)

    # Lakukan prediksi
    prediction = model.predict(new_data_scaled)
    
    # --- PERUBAHAN DI SINI ---
    # Gunakan np.maximum(0, array) untuk mengubah semua nilai negatif menjadi 0
    adjusted_prediction = np.maximum(0, prediction[0])
    # --- AKHIR PERUBAHAN ---
    
    # Format hasil prediksi dengan menggunakan nilai yang sudah disesuaikan
    predicted_nutrients = {
        'Carbohydrates': f"{adjusted_prediction[0]:.2f} g",
        'Protein':       f"{adjusted_prediction[1]:.2f} g",
        'Fat':           f"{adjusted_prediction[2]:.2f} g",
        'Calories':      f"{adjusted_prediction[3]:.0f} kkal"
    }
    
    print(f"\nPrediksi untuk '{food_type_str}' dengan porsi {portion_size_int} gram:")
    for nutrient, value in predicted_nutrients.items():
        print(f"  - Estimasi {nutrient}: {value}")
        
    return predicted_nutrients

--- MEMULAI BAGIAN I: PELATIHAN DAN PENYIMPANAN MODEL SVR ---
Dataset 'food_data_final_shuffled.csv' berhasil dimuat.

Fitur Input (X): ['food_type', 'portion_size']
Target Output (y): ['carbohydrates', 'protein', 'fat', 'calories']

--- Training Multi-Output SVR ---
SVR training complete.

--- Mengevaluasi Model SVR ---

Hasil Evaluasi SVR per Target Nutrisi:
                     MAE       RMSE  R2 Score   MAPE
carbohydrates   1.615093   3.093904  0.981108  9.87%
protein         1.264270   2.134584  0.982037  9.39%
fat             1.080985   1.624181  0.985179  9.16%
calories       15.062162  22.072878  0.987206  6.57%

--- Menyimpan model SVR, scaler, dan nama fitur ---
Model dan komponen berhasil disimpan ke dalam file .pkl


--- MEMULAI BAGIAN II: PREDIKSI PADA DATA BARU ---


In [43]:
# Contoh running model estimasi SVR

predict_nutrition_from_portion(food_type_str='Steak Sapi', portion_size_int=190)


Prediksi untuk 'Steak Sapi' dengan porsi 190 gram:
  - Estimasi Carbohydrates: 2.91 g
  - Estimasi Protein: 48.65 g
  - Estimasi Fat: 36.83 g
  - Estimasi Calories: 522 kkal


{'Carbohydrates': '2.91 g',
 'Protein': '48.65 g',
 'Fat': '36.83 g',
 'Calories': '522 kkal'}